In [2]:
def column_fill_up(data, column_name):
    nan = data[column_name].isna()
    cat = data[column_name]
    lens = len(nan)
    for index in range(lens):
        if nan[index] == True:
            data.loc[index, column_name] = cat[index-1]
    return data
    

In [12]:
import datetime 
now = datetime.datetime.now()
last_month = now.month - 1 if now.month > 1 else 12
#month = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()[last_month-1]
month = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()[last_month]
#path = r"C:\Users\admin\Downloads\Primary Cell Product Order and Status - {} 2021.xlsx".format(month) - Windows 
path = r"/Users/weizhang/Downloads/Primary Cell Product Order and Status - {} 2021.xlsx".format(month)

In [10]:
import pandas as pd 

In [13]:
columns = ['Date', 'PO#','Order Number', 'Customer company', "New customer", 'Distributor', 'Cat. #', 'Description', 'Quantity ', 'Completed']
raw_df = pd.read_excel(path, usecols=columns)

In [23]:
df_dropna = raw_df.dropna(how='all', subset=['Quantity '], axis=0).reset_index().drop('index', axis=1)

In [26]:
df_dropna.tail(10)

,Date,Order Number,PO#,Customer company,New customer,Distributor,Cat. #,Description,Quantity,Completed
112,2021-10-28 00:00:00,SO2103993,4513524471,TX UNIV OF HSC HOUSTON,No,VWR,10HU-003-CR50M,"Human PBMCs (Cryopreserved)-Cryopreserved, 50 ...",1.0,Yes
113,NaN,SO2103994,22438,Caltag Medsystems Ltd.-United Kingdom(Net30),No,None,40HU-005-4M,"Human Motor Neurons (iPSC-derived, Normal)-Cry...",5.0,Yes
114,NaN,NaN,NaN,NaN,No,None,MD-0022-100ML,Motor Neuron Maintenance Medium-100 mL,2.0,Yes
115,NaN,NaN,NaN,NaN,No,None,MD-0039-500ML,Astrocyte Medium-500 mL,1.0,Yes
116,NaN,SO2103995,cc-13008,"Prime Medicine, Inc",No,None,MD-0022-100ML,Motor Neuron Maintenance Medium-100 mL,1.0,Delivery date scheduled
117,NaN,NaN,NaN,NaN,No,None,40HU-005-1M,"Human Motor Neurons (iPSC-derived, Normal)-Cry...",2.0,Delivery date scheduled
118,NaN,SO2103996,8508887,Merck Sharp & Dohme Corp,No,Fisher Scientific,MD-0011-500ML,Fibroblast Growth Medium-500 mL,5.0,Delivery date scheduled
119,NaN,SO2103997,4513525798,"Janssen Research & Development, LLC",No,VWR,10HU-003-CR100M,"Human PBMCs (Cryopreserved)-Cryopreserved, 100...",35.0,Delivery date scheduled
120,2021-12-29 00:00:00,SO2103998,E003513119,"Synthorx, INC.",No,None,CXPR-10,"Custom Whole Blood Sample Processing ((LSR, 10...",2.0,Delivery date scheduled
121,NaN,SO2103999,2021-359,"Prometheus Biosciences, Inc.",No,None,CXPR-15-200M,"Human PBMC (Fresh)-Fresh Human PBMC, 200-250 m...",3.0,Delivery date scheduled


In [ ]:
import pandas as pd
import datetime

def column_fill_up(data, column_name):
    nan = data[column_name].isna()
    cat = data[column_name]
    lens = len(nan)
    for index in range(lens):
        if nan[index] == True:
            data.loc[index, column_name] = cat[index-1]
    return data

def get_monthly_data(path, path1, product_name):
    
    # import data
    df1 = pd.read_excel(path, usecols=['Date', 'PO#','Order Number', 'Customer company', 'Distributor', 'Cat. #', 'Description', 'Quantity ', 'Completed'])
    
    item_name = pd.read_excel(product_name, usecols=['Cat. #', 'Product Name', 'Category Name'])

    # fill up company name
    columns = ['Customer company', 'PO#', 'Distributor', 'Order Number', 'Cat. #', 'Description', 'Date']
    for item in columns:
        df2 = column_fill_up(df1, item)
        df1 = df2
    
    # dropna - drop services rows and empty rows (not values for column Quantity and Completed)
    df2 = df1.dropna(how='all', subset=['Quantity '], axis=0).reset_index().drop('index', axis=1)

    # Remove pending, replacement, and cancelled orders
    df3 = df2[(df2['Completed'] == 'Yes') | (df2['Completed'] == 'Delivery date scheduled')]

    # backordered 
    df4 = df2[df2['Completed'] == 'Pending']

    # total sales summary
    result = df3.groupby('Cat. #')['Quantity '].sum().sort_values(ascending=False).reset_index()
    result_2 = result.merge(item_name, how='left', on='Cat. #')

    #path1 = r'C:\Users\admin\iXCells Biotechnologies\Product Development - Documents\Product Lists\Inventory and Sales Records\Sales\2021 Sales\{}2021 Sales Records.xlsx'.format(month)
    datatoexcel = pd.ExcelWriter(path1, engine='xlsxwriter')
    result_2.to_excel(datatoexcel, sheet_name='Sales Summary', index=False)
    df3.to_excel(datatoexcel, sheet_name='Completed Sales', index=False)
    df4.to_excel(datatoexcel, sheet_name='Pending', index=False)
    df2.to_excel(datatoexcel, sheet_name='Sales Records_data', index=False)



    datatoexcel.save()

# https://docs.google.com/spreadsheets/d/1BNtwHjC81NS4UThi1QABrlvpoNFTSuILSxG8dmSSk3A/export?format=xlsx&gid=439584568 download one sheet

# get the name of last month 
import datetime
now = datetime.datetime.now()
last_month = now.month-1 if now.month > 1 else 12
month = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()[last_month-1]



path1 = r'C:\Users\admin\iXCells Biotechnologies\Product Development - Documents\Product Lists\Inventory and Sales Records\Sales\2021 Sales\{}2021 Sales Records.xlsx'.format(month)
path = r"C:\Users\admin\Downloads\Primary Cell Product Order and Status - {} 2021.xlsx".format(month)
product_name = r"C:\Users\admin\OneDrive - iXCells Biotechnologies\Projects\Clean Data Backup\Website Product Name.xlsx"
get_monthly_data(path, path1, product_name)